In [1]:
# Validate models

In [2]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def create_target_columns_to_classification(df):
    df = df.copy()
    target_cols = ['home_win_flag_base_pred', 'draw_flag_base_pred','away_win_flag_base_pred']
    target_cols_dict = {'home_win_flag_base_pred':0,'draw_flag_base_pred':1,'away_win_flag_base_pred':2}
    def get_y(row):
        for col in target_cols:
            if row[col] == 1:

                y = target_cols_dict[col]
            else:
                y = 0
        return y
    df['y'] = df[target_cols].apply(get_y,axis=1)
    df['y'] = df['y'].astype('category')
    return df

def get_pred_targets(x,colname):
    value = target_cols_dict[colname]
    a = 0
    if value == x:
        a = 1
    
    return a
target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
target_cols = ['home_win_flag','draw_flag','away_win_flag']

In [4]:
df = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
#keep_seasons = ['2019-2020','2020-2021','2021-2022']
#df = df[df.season.isin(keep_seasons)]


df.head()
df = df.reset_index(drop=True)

target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
#features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

In [5]:
#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=34)
X_test

,odds_home_team_win,odds_draw,odds_away_team_win,home_team_match_nr,home_team_goal_roll1_sum,home_team_goal_roll2_sum,home_team_goal_roll3_sum,home_team_goal_roll4_sum,home_team_opponents_goal_roll1_sum,home_team_opponents_goal_roll2_sum,...,odds_away_team_draw_roll1_mean,odds_away_team_draw_roll2_mean,odds_away_team_draw_roll3_mean,odds_away_team_draw_roll4_mean,odds_away_team_defeat_roll1_mean,odds_away_team_defeat_roll2_mean,odds_away_team_defeat_roll3_mean,odds_away_team_defeat_roll4_mean,away_team_point_mean,away_team_expected_point_mean
62088,4.03,3.68,1.87,28.0,2.0,2.0,4.0,5.0,1.0,1.0,...,3.72,3.735,3.703,3.715,5.06,3.480,3.597,3.610,1.154,2.308
24707,3.10,3.30,2.50,14.0,1.0,2.0,4.0,6.0,1.0,3.0,...,3.60,3.450,3.400,3.450,3.80,3.400,3.367,3.400,2.333,0.500
46709,2.22,3.19,3.44,34.0,1.0,2.0,5.0,7.0,1.0,1.0,...,3.29,3.370,3.377,3.295,3.89,3.685,3.373,3.432,1.235,1.059
7094,1.25,5.50,12.00,13.0,1.0,4.0,7.0,9.0,2.0,5.0,...,3.25,4.375,4.017,3.812,2.38,1.815,2.543,2.595,1.200,0.000
47838,2.40,3.37,2.83,27.0,1.0,1.0,5.0,8.0,2.0,3.0,...,3.31,3.300,3.303,3.295,4.61,3.770,3.360,3.120,1.083,0.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41952,1.72,3.75,5.50,31.0,3.0,4.0,5.0,5.0,0.0,2.0,...,3.30,3.350,3.333,3.350,3.50,2.850,2.733,2.575,1.133,0.400
41130,3.50,3.30,2.30,22.0,0.0,2.0,2.0,5.0,2.0,3.0,...,3.30,3.525,3.483,3.588,3.30,3.750,3.400,3.632,1.455,2.727
2158,1.73,3.40,4.33,5.0,3.0,6.0,7.0,8.0,4.0,6.0,...,3.40,3.300,3.367,3.400,4.00,2.955,2.547,3.160,0.000,0.000
22138,2.40,3.20,3.00,10.0,0.0,2.0,3.0,3.0,2.0,3.0,...,3.30,3.250,3.417,3.512,2.20,2.250,3.500,3.018,0.500,0.000


In [6]:
# Load models

In [7]:
# load model
p_home_win = pickle.load(open('pipe_home_win_base_model_3.p', "rb"))
p_draw =     pickle.load(open('pipe_p_draw_base_model_3.p', "rb"))
p_away_win =  pickle.load(open('pipe_p_away_win_base_model_3.p', "rb"))
xgb_m =      pickle.load(open('xgb_m_3.p', "rb"))

In [8]:
for x in range(12):
    df = df.reset_index(drop=True)
    X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=76 + x)
    val_df = pd.DataFrame(y_test)
    val_df.columns = ['home_win_flag']
    val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
    val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
    val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
    val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
    val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
    val_df[['home_win_prob_0','home_win_prob_1']] = p_home_win.predict_proba(X_test)
    val_df[['away_win_prob_0','away_win_prob_1']] = p_away_win.predict_proba(X_test)
    for y in target_cols:    
        fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_base_pred'])
        print(f' {y} auc is: {auc(fpr, tpr)}')
    val_df = create_target_columns_to_classification(val_df)
    retrain_index = val_df[(val_df.home_win_flag_base_pred ==1) & (val_df.home_win_prob_1 < 0.75)].index.values
    val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(X_test.loc[retrain_index][features_cols])
    val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)
    for t in target_cols:
        val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
    print('           ...                 ')
    for y in target_cols:    
        fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
        print(f'modified {y} auc is: {auc(fpr, tpr)}')
    print('-------------------------------------------------------------')

 home_win_flag auc is: 0.6013378136782982
 draw_flag auc is: 0.5
 away_win_flag auc is: 0.6204518721713973
           ...                 
modified home_win_flag auc is: 0.6086694666301511
modified draw_flag auc is: 0.5027584362616121
modified away_win_flag auc is: 0.6237478171093619
-------------------------------------------------------------
 home_win_flag auc is: 0.5996398542691941
 draw_flag auc is: 0.5
 away_win_flag auc is: 0.6173066126076063
           ...                 
modified home_win_flag auc is: 0.6068585172908054
modified draw_flag auc is: 0.5019065837002457
modified away_win_flag auc is: 0.6206756868388517
-------------------------------------------------------------
 home_win_flag auc is: 0.6096253656043263
 draw_flag auc is: 0.5
 away_win_flag auc is: 0.6241046830174329
           ...                 
modified home_win_flag auc is: 0.6160232937218431
modified draw_flag auc is: 0.5017055938288871
modified away_win_flag auc is: 0.6270959497318706
---------------------

In [9]:
f = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
#keep_seasons = ['2019-2020','2020-2021','2021-2022']
#df = df[df.season.isin(keep_seasons)]
df = df.reset_index(drop=True)

df.head()

target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
#features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=77)
val_df = pd.DataFrame(y_test)
val_df.columns = ['home_win_flag']
val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
val_df[['home_win_prob_0','home_win_prob_1']] = p_home_win.predict_proba(X_test)
val_df = create_target_columns_to_classification(val_df)
#retrain_index = val_df[(val_df.home_win_flag_base_pred != val_df.away_win_flag)].index.values
retrain_index = val_df[(val_df.home_win_flag_base_pred ==1) & (val_df.home_win_prob_1 < 0.75)].index.values
val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(X_test.loc[retrain_index][features_cols])
val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)
for t in target_cols:
    val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

 home_win_flag auc is: 0.6068585172908054
 draw_flag auc is: 0.5019065837002457
 away_win_flag auc is: 0.6206756868388517


In [10]:
val_df

,home_win_flag,draw_flag,away_win_flag,home_win_flag_base_pred,draw_flag_base_pred,away_win_flag_base_pred,home_win_prob_0,home_win_prob_1,y,final_pred,home_win_flag_pred,draw_flag_pred,away_win_flag_pred
27378,0.0,0.0,1.0,1.0,0.0,0.0,0.452342,0.547658,0,0.0,1,0,0
62582,0.0,0.0,1.0,1.0,0.0,0.0,0.390819,0.609181,0,0.0,1,0,0
31885,0.0,0.0,1.0,1.0,0.0,0.0,0.483070,0.516930,0,2.0,0,0,1
9235,1.0,0.0,0.0,1.0,0.0,0.0,0.402983,0.597017,0,0.0,1,0,0
1982,0.0,0.0,1.0,1.0,0.0,0.0,0.489026,0.510974,0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708,0.0,0.0,1.0,0.0,0.0,1.0,0.566695,0.433305,2,2.0,0,0,1
37443,1.0,0.0,0.0,1.0,0.0,0.0,0.332620,0.667380,0,0.0,1,0,0
11502,1.0,0.0,0.0,1.0,0.0,0.0,0.175750,0.824250,0,0.0,1,0,0
62033,0.0,0.0,1.0,1.0,0.0,0.0,0.292467,0.707533,0,0.0,1,0,0
